In [5]:
import pandas as pd 
import os
import glob
import psycopg2
from sql_queries import *

In [19]:
try:
    conn = psycopg2.connect('host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres')
    cur = conn.cursor()
except psycopg2.Error as e:
    print(e)

In [3]:
def get_files(dir):
    all_files = []
    for root, dirs, files in os.walk(dir):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))

    return all_files

In [4]:
filepath =  'data/song_data'
song_files = get_files(filepath)

song_files[0]


'/home/indocyber/course/Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [5]:
df = pd.read_json(song_files[0],lines=True)

df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [6]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]

song_data

array(['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0,
       218.93179], dtype=object)

In [23]:
cur.execute(insert_table_songs, song_data)
conn.commit()

In [6]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0]

artist_data

array(['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan],
      dtype=object)

In [7]:
cur.execute(insert_table_artists, artist_data)
conn.commit()

In [6]:
filepath = 'data/log_data'
log_files = get_files(filepath)

log_files[0]

'/home/indocyber/course/Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-01-events.json'

In [7]:
df = pd.read_json(log_files[0],lines=True)

df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [8]:
df = df[df.page == 'NextSong']

df.dtypes

artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration       int64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId             int64
dtype: object

In [9]:
t = df.copy()

t['ts'] = pd.to_datetime(t['ts'],unit='ms')

# print(t.ts, t.ts.dt.hour, t.ts.dt.day, t.ts.dt.dayofweek, t.ts.dt.month, t.ts.dt.year, t.ts.dt.weekday)

In [10]:
time_data = (t.ts, t.ts.dt.hour, t.ts.dt.day, t.ts.dt.dayofweek, t.ts.dt.month, t.ts.dt.year, t.ts.dt.weekday)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [11]:
time_dict = {"start_time" : t.ts, 
             "hour" : t.ts.dt.hour, 
             "day" : t.ts.dt.day, 
             "week" : t.ts.dt.dayofweek, 
             "month" : t.ts.dt.month, 
             "year" : t.ts.dt.year, 
             "weekday" : t.ts.dt.weekday}

time_df = pd.DataFrame.from_dict(time_dict)
time_df.head()

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,3,11,2018,3
4,2018-11-01 21:05:52.796,21,1,3,11,2018,3
5,2018-11-01 21:08:16.796,21,1,3,11,2018,3
6,2018-11-01 21:11:13.796,21,1,3,11,2018,3
7,2018-11-01 21:17:33.796,21,1,3,11,2018,3


In [13]:
for i, row in time_df.iterrows():
    cur.execute(insert_table_time, list(row))
    conn.commit()

In [12]:
user_data = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

user_data

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
8,8,Kaylee,Summers,F,free
9,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
13,26,Ryan,Smith,M,free


In [16]:
for i, row in user_data.iterrows():
    cur.execute(insert_table_users,list(row))
    conn.commit()

In [13]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [22]:
for i, row in df.iterrows():
    result = cur.execute("select a.song_id, b.artist_id from songs a inner join artists b on a.artist_id = b.artist_id where a.title = %s and b.name = %s and a.duration = %s", (row.song, row.artist, row.length))
    songid, artistid = result if result else None, None

    starttime = pd.to_datetime(row.ts,unit='ms')

    cur.execute(insert_table_songplays,(i, starttime, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent))
    conn.commit()



In [23]:
conn.close()

In [6]:
def process_data(filepath):

    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, "*.json"))
        for f in files:
            all_files.append(os.path.abspath(f))

    num_files = len(all_files)
    print("{} files found in {}".format(num_files, filepath))

In [10]:
process_data("data/song_data")
process_data("data/log_data")

83 files found in data/song_data
30 files found in data/log_data
